In [ ]:
from jupyter_plotly_dash import JupyterDash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
import numpy as np
import os
from sys import path as syspath
syspath.append(os.path.expanduser("~/srdjan_functs/"))
test=False

%load_ext autoreload
%autoreload 1
%aimport Regions, physio_def_1, numeric

from physio_def_1 import *
from numeric import *

from Regions import Regions
from numeric import *

from plotly.subplots import make_subplots

from sys import exc_info

from plotly import colors
os.chdir(os.path.expanduser("~"))

In [24]:
myfig = plotROIs(regions,indices=[0])

/opt/tljh/user/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:35: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.



In [30]:
[(d.type, d.xaxis) for d in myfig.data]

[('heatmap', 'x'),
 ('heatmap', 'x2'),
 ('scatter', 'x'),
 ('scatter', 'x2'),
 ('scatter', 'x'),
 ('scatter', 'x2')]

In [22]:
def plotROIs(regions,indices=None):
    if indices is None:
        indices = regions.df.sort_values("size",ascending=False).index[:10]
    fig1 = make_subplots(rows=1, cols=2, horizontal_spacing=0.02,
    #                      shared_xaxes=True,
                         shared_yaxes=True,
                    subplot_titles=("Standard Deviation","Template"))
    for j,im in enumerate([regions.std_image, regions.image]):
        fig1.add_heatmap(z=np.array(im),col=j+1,row=1, hoverinfo='skip',showscale=False)
    fig1.update_layout(
        height = 450,
        width = 900,
    )

    if len(indices)>0:

        ia = 0
        for i in indices:
            cl = colors.DEFAULT_PLOTLY_COLORS[ia%10]
            bds = regions.df.loc[i,"boundary"]
            bds += [bds[0]]
            y,x = np.array(bds).T
            for c in [1,2]:
                ln = go.Scatter(x=x,y=y,
                                line=dict(width=1,color=cl),
                                mode="lines",
                                showlegend = c==1,
                                name = str(i),
                                hoverinfo='text',
                                hovertext=["%i"%(i)]*len(bds),
                             )
                fig1.add_trace(ln,col=c,row=1)
            ia += 1

        y,x = np.vstack(np.vstack(regions.df.pixels[i]).mean(axis=0) for i in indices).T

        pts = go.Scatter(x=x,y=y,
                        mode="markers",
                        showlegend = False,
                        opacity=0.0,
                         name=str(i),
                        marker=dict(color=np.arange(len(x))%10),
                        hovertext=["%i"%(i) for i in indices],
                        hoverinfo="text"
                     )
        for c in [1,2]: fig1.add_trace(pts,col=c,row=1)
    
    for c in [1,2]:
        fig1.update_xaxes({"range":[-.5,im.shape[1]-.5]},col=c,row=1)
        fig1.update_yaxes({"range":[-.5,im.shape[0]-.5]},col=c,row=1)
    return fig1

def parseRoiChoices(roi_mode, roi_number):
    try:
        roi_number = int(roi_number)
    except:
        roi_number = 3
    if roi_number<1:
        roi_number=1
    if roi_number>20:
        roi_number=20
    if roi_mode=="largest":
        ix = regions.df.sort_values("size",ascending=False).index[:roi_number]
    if roi_mode=="rnd":
        ix = np.random.choice(regions.df.index, roi_number)
    return ix

def plotTraces(regions,indices=None):
    if indices is None:
        indices = regions.df.sort_values("size",ascending=False).index[:10]
    fig = go.Figure()
    for i in indices:
        fig.add_trace(go.Scatter(
            x=regions.time,
            y=regions.df.loc[i,"trace"],
            line=dict(width=1),
            mode="lines",
            name=str(i),
#             text=[str(i)]*len(regions.time),
#             hoveinfo="name"
        ))
    fig.update_layout(
        hovermode="x",
       xaxis_title='time [s]',
       yaxis_title='intensity')
    return fig

In [1]:
# %%capture
from caiman import movie as cmovie

## global vars
npzFile = None
metadata = None
movie = None
regions = None

app = JupyterDash(__name__,
                  width=800,
#                   height=1000,
                 )
app.layout = html.Div(children=[
    html.H1(children='CaRec Viewer'),
    html.Div([
        html.P("At your disposal."),
    ]),

    ####################################
    html.H2(children='Importing'),
    html.Div([
        html.P('Enter path to npz (e.g. local_data/testdir/Experiment38a_7_Series016.npz), and press enter.'),
        html.P('Loading can take a few seconds [for very large files even a minute].'),
    ]),
    dcc.Input(
            id="npz_filename",
            type="text",
            placeholder="local_data/testdir/Experiment38a_7_Series016.npz",
            debounce=True,
            size = 50,
            value="local_data/testdir/Experiment38a_7_Series016.npz" if test else None,
        ),
    html.Div(id="npz_feedback", children=""),

    ####################################
    html.H2(children='Resampling'),
    html.Div(children="""
             Resample movie if you wish. It is optional, but if you do it, the analysis downstream goes much faster.
             Enter the new desired frequency in Hz (e.g. 2), and press enter.
             """),
    dcc.Input(
            id="resample_input",
            type="text",
            placeholder="2",
            debounce=True,
            size = 10,
            value="2" if test else None,
        ),
    html.Div(id="resample_feedback", children=""),
    
    ####################################
    html.H2(children='Create ROIs'),
    html.Div([
        html.P("For the proper analysis, you need to set the size of the spatial filter."),
        html.P("It's value should be of the order of the  half of the typical cell dimension (in pixels!)."),
        html.P("In many of our recording, pixel size is around 2 µm. If we consider a typical cell of 10 µm, filter should be around 10/2/2 = 2.5. It needs to be an integer, so reasonable numbers to put would be 2 or 3, perhaps even 4. You can change it later."),
    ]),
    dcc.Input(
        id="gSig_filt",
        type="text",
        placeholder="2",
#         debounce=True,
        size = 10,
        value="2" if test else None,
    ),
    
    html.Div(id="roi_feedback"),
    
    html.H2(children='ROIs'),
    dcc.RadioItems(
        id="roi_choice",
        options=[
            {'label': 'Largest', 'value': 'largest'},
            {'label': 'Random', 'value': 'rnd'},
            # {'label': 'Random, larger than', 'value': 'rnd_th'},
        ],
        value='largest'),
    dcc.Input(
            id="roi_choice_number",
            type="text",
            placeholder="10",
            debounce=True,
            size = 10,
            value='10' if test else None
        ),
    html.Div(id="roi_choice_feedback", children=""),
    
    html.H3(children='Images'),
    
    dcc.Graph(id='images'),
    
    html.H3(children='Traces'),
    
    dcc.Graph(id='roi_traces_raw',),

])

@app.callback(Output("npz_feedback", "children"),
              [Input("npz_filename", "value")],)
def loadMovie(val):
    try:
        from pandas import read_csv
        global npzFile, movie, metadata
        npzFile = val
        metadata = read_csv(npzFile.replace("npz","txt")).loc[0]
        if not hasattr(metadata,"freq"):
            metadata.freq = 1
        movie = import_npz_files([npzFile])
        movie = movie.astype("float")
        if len(movie.shape)==2:        
            movie = movie.reshape((1,-1))
        movie = cmovie(movie)
        movie.fr = metadata.freq
        feedback = [html.P(f"Recording {npzFile} has {metadata['T']} timeframes and is done at {metadata.freq:.4} Hz, with a pixel size of {metadata.pxSize:.3} µm")]
        if metadata.freq>3:
            feedback += [html.P("This is probably a lot higher frequency than you need, consider resampling to lower frequency, e.g. 2 Hz")]
        return feedback
    except:
        return exc_info()[0]

@app.callback(Output("resample_feedback", "children"),
              [Input("resample_input", "value")],)
def resample(val):
    global movie
    try:
        newFreq = float(val)
        n_rebin = int(np.ceil(movie.fr/newFreq))
        movie = rebin(movie,n_rebin)
        movie.fr = movie.fr/n_rebin
        feedback = f"Resampling finished. New frequency is {movie.fr:.4} Hz."
        return feedback
    except:
        return exc_info()[0]

@app.callback(
    Output("roi_feedback", "children"),
    [Input("gSig_filt", "value")]
)
def calcRois(val):
    global regions
    gSig_filt = int(val)
    gSig_filt = (gSig_filt,gSig_filt)
    regions = Regions(movie,th=0,gSig_filt=gSig_filt)
    regions.purge_lones(5)
    C = regions.df
    C.drop(index=C.query("size<7").index,inplace=True)
    out = f"{len(regions.df)} rois discovered."
    regions.calcTraces()
    return out

@app.callback(
    Output("roi_choice_feedback", "children"),
    [Input("roi_choice", "value"), Input("roi_choice_number", "value")]
)
def showRoiChoices(roi_mode,roi_number):
    ix = parseRoiChoices(roi_mode,roi_number)
    output = ",".join([str(i) for i in ix])
    return output

@app.callback(
    Output("roi_traces_raw", "figure"),
#     [Input("roi_choice_feedback", "value")]
    [Input("roi_choice", "value"), Input("roi_choice_number", "value")]
)

def showTraces(roi_mode,roi_number):
    global regions
    ix = parseRoiChoices(roi_mode,roi_number)
#     ix = list(map(int,val.split(",")))
    fig = plotTraces(regions,indices=ix)
    return fig

@app.callback(
    Output("images", "figure"),
    [Input("roi_choice", "value"), Input("roi_choice_number", "value")]
)
def addRoisToStatFigs(roi_mode,roi_number):
    global regions
    ix = parseRoiChoices(roi_mode,roi_number)
    fig = plotROIs(regions,ix)
    return fig



link2app = "https://ctn.physiologie.meduniwien.ac.at"+app.get_app_root_url()

/opt/tljh/user/envs/physio/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.decomposition.incremental_pca module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.decomposition. Anything that cannot be imported from sklearn.decomposition is now part of the private API.



In [5]:
myfig = plotROIs(regions,indices=[0])

/opt/tljh/user/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:59: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.



In [2]:
from IPython.core.display import HTML
HTML(f'<a href="{link2app}">{link2app}</a>')

In [3]:
app